## 필요 라이브러리 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!apt-get install python3-openslide

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (69.3 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import shutil
import time
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
from PIL import Image
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split

!pip install ipython-autotime

%load_ext autotime
%tensorflow_version 2.x
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 36.7 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
time: 1.96 s (started: 2022-11-22 05:35:01 +00:00)


In [ ]:
##임포트(in 주피터노트북)
# OPENSLIDE_PATH = r'C:/cakd7/3차 프로젝트/openslide-win64-20220811/bin'

# import os
# if hasattr(os, 'add_dll_directory'):
#     # Python >= 3.8 on Windows
#     with os.add_dll_directory(OPENSLIDE_PATH):
#         import openslide
# else:
#     import openslide
    
    
# from openslide.deepzoom import DeepZoomGenerator
# import numpy as np
# import matplotlib.pyplot as plt

# import cv2
# import os

# 이미지 패치화하기

## TEST: 표본 한개만 진행

### 이미지 패치 추출

In [ ]:
# 마스킹
slide_mask = openslide.open_slide('./train_imgs_mask_tiff/BC_01_0015.tiff')

# 원본
slide_real = openslide.open_slide('./train_imgs_tiff/BC_01_0015.tiff')


## 확인
slide_mask, slide_real

(ImageSlide('./train_imgs_mask_tiff/BC_01_0015.tiff'),
 ImageSlide('./train_imgs_tiff/BC_01_0015.tiff'))

In [ ]:
# level_dimensions: Deep Zoom 수준에 대한 튜플 목록
dims_mask = slide_mask.level_dimensions
dims_real = slide_real.level_dimensions

dims_mask, dims_real

(((5311, 2308),), ((5311, 2308),))

In [ ]:
# 50*50 픽셀 타일로 쪼개기
# 마스킹
tiles_mask = DeepZoomGenerator(slide_mask, tile_size = 50, overlap = 0, limit_bounds = False)

# 원본
tiles_real = DeepZoomGenerator(slide_real, tile_size = 50, overlap = 0, limit_bounds = False)


# 타일 개수 확인
tiles_mask.tile_count, tiles_real.tile_count      

(6771, 6771)

In [ ]:
# 해상도 레벨 확인(작을수록 고해상도인 것 같음, 근데 작을수록 아래의 col, row개수가 적어짐..)

mask_level = tiles_mask.level_count
real_level = tiles_real.level_count

mask_level, real_level

(14, 14)

In [ ]:
# 타일의 열개수, 행개수 객체화
cols_mask, rows_mask = tiles_mask.level_tiles[mask_level - 1]
cols_real, rows_real = tiles_real.level_tiles[real_level - 1]

cols_mask, rows_mask, cols_real, rows_real

(107, 47, 107, 47)

### 폴더에 패치 파일 저장

In [ ]:
# 타일로 쪼개서 저장할 디렉토리 생성
tile_real_normal_dir = '원본데이터_타일/split_real_normal/'
tile_real_abnormal_dir = '원본데이터_타일/split_real_abnormal/'
tile_mask_normal_dir = '원본데이터_타일/split_mask_normal/'
tile_mask_abnormal_dir = '원본데이터_타일/split_mask_abnormal/'

# 원본데이터와 마스킹 데이터 크기가 같아서 결론적으로 row, col개수가 같을 것으로 보임
# 해상도 레벨도 마찬가지
# np.set_printoptions(threshold=np.inf, linewidth=np.inf) 

for row in range(rows_mask):
    for col in range(cols_mask):
        
        # 타일 이름 정해주기
        tile_name_real_normal = os.path.join(tile_real_normal_dir, f'원본_{row}_{col}')
        tile_name_real_abnormal = os.path.join(tile_real_abnormal_dir, f'원본_{row}_{col}')
        tile_name_mask_normal = os.path.join(tile_mask_normal_dir, f'원본_{row}_{col}')
        tile_name_mask_abnormal = os.path.join(tile_mask_abnormal_dir, f'원본_{row}_{col}')
        print('지금 저장 중인 파일: ', '\n', 'tile_name: ', row,"_", col)
        
        # 해당 컬럼, 로우에 해당하는 타일 객체화
        ## 마스킹
        temp_tile_mask = tiles_mask.get_tile(mask_level - 1, (col, row))
#         print(temp_tile_mask)
        ## 원본
        temp_tile_real = tiles_real.get_tile(mask_level - 1, (col, row))
#         print(temp_tile_real)
        
        temp_tile_real_RGB = temp_tile_real.convert('RGB')

        # 해당 타일 이미지를 3차원 배열화-> rgb로 3차원이나옴
        ## 마스킹
        temp_tile_mask_np = np.array(temp_tile_mask)
#         print(temp_tile_mask_np.shape)
        ## 원본
        temp_tile_real_np = np.array(temp_tile_real_RGB)
#         print(temp_tile_real_np.shape)
        
    
        # 타일의 로우, 컬럼 수 객체화
        tile_row, tile_col, dim = temp_tile_real_np.shape
        
        # 차원값 모두 255인(흰색부분)의 픽셀 수 및 비율 확인, 정상세포 픽셀 수 및 비율 확인
        white = 0
        normal = 0
        for i in range(tile_row):
            for j in range(tile_col):
                if temp_tile_mask_np[i, j, 0] == 255 and temp_tile_mask_np[i, j, 1] == 255 and temp_tile_mask_np[i, j, 2] == 255:
                    white += 1
                    
                elif temp_tile_mask_np[i, j, 0] == 225 and temp_tile_mask_np[i, j, 1] == 225 and temp_tile_mask_np[i, j, 2] == 225:
                    normal += 1
                    
        white_ratio = white / 2500
        normal_ratio = normal / 2500
        
        print(white_ratio)
        print(normal_ratio)
        
        if white_ratio > 0.3: 
            pass
        elif normal_ratio > 0.05 and tile_row == 50 and tile_col == 50 :
            # 저장하기
            plt.imsave(tile_name_real_normal + ".png", temp_tile_real_np)
            plt.imsave(tile_name_mask_normal + ".png", temp_tile_mask_np)
            
        elif tile_row == 50 and tile_col == 50:
            # 저장하기
            plt.imsave(tile_name_real_abnormal + ".png", temp_tile_real_np)
            plt.imsave(tile_name_mask_abnormal + ".png", temp_tile_mask_np)

지금 저장 중인 파일:  
 tile_name:  0 _ 0
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 1
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 2
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 3
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 4
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 5
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 6
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 7
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 8
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 9
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 10
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 11
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 12
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 13
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 14
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 15
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 16
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 17
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 18
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 19
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 20
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 21
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  0 _ 22
1.0
0.0
지금 저장 중인 파일:  
 tile_

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 92
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 93
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 94
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 95
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 96
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 97
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 98
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 99
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 100
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 101
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 102
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 103
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 104
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 105
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  1 _ 106
0.22
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 0
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 1
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 2
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 3
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 4
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 5
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 6
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  2 _ 7
1.0
0.0
지금 

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 77
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 78
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 79
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 80
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 81
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 82
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 83
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 84
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 85
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 86
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 87
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 88
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 89
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 90
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 91
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 92
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 93
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 94
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 95
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 96
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 97
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 98
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  3 _ 99
1.0
0.0
지금 

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 66
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 67
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 68
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 69
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 70
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 71
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 72
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 73
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 74
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 75
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 76
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 77
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 78
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 79
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 80
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 81
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 82
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 83
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 84
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 85
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 86
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 87
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  5 _ 88
1.0
0.0
지금 

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 52
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 53
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 54
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 55
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 56
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 57
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 58
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 59
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 60
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 61
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 62
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 63
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 64
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 65
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 66
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 67
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 68
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 69
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 70
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 71
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 72
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 73
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  7 _ 74
1.0
0.0
지금 

...

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 35
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 36
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 37
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 38
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 39
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 40
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 41
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 42
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 43
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 44
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 45
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 46
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 47
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 48
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 49
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 50
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 51
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 52
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 53
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 54
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 55
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  37 _ 56
1.0
0.0
지금 저장 중인 파일:  
 tile_nam

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 18
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 19
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 20
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 21
0.7504
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 22
0.108
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 23
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 24
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 25
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 26
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 27
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 28
0.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 29
0.2552
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 30
0.9596
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 31
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 32
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 33
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 34
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 35
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 36
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 37
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 38
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  39 _ 39
1.0
0.0
지금 저장 중인 파일: 

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 5
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 6
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 7
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 8
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 9
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 10
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 11
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 12
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 13
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 14
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 15
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 16
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 17
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 18
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 19
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 20
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 21
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 22
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 23
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 24
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 25
0.8724
0.0
지금 저장 중인 파일:  
 tile_name:  41 _ 26
0.3888
0.0
지금 저장 중인 파일:  
 tile_na

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  42 _ 102
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  42 _ 103
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  42 _ 104
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  42 _ 105
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  42 _ 106
0.22
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 0
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 1
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 2
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 3
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 4
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 5
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 6
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 7
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 8
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 9
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 10
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 11
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 12
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 13
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 14
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 15
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  43 _ 16
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  

1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 88
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 89
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 90
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 91
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 92
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 93
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 94
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 95
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 96
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 97
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 98
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 99
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 100
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 101
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 102
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 103
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 104
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 105
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  44 _ 106
0.22
0.0
지금 저장 중인 파일:  
 tile_name:  45 _ 0
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  45 _ 1
1.0
0.0
지금 저장 중인 파일:  
 tile_name:  45 _ 2
1.0
0.0
지금 저장 중인 파일:  
 til

## TEST: 마스킹 데이터 진행

### png 파일을 tiff파일로 변환

In [ ]:
from PIL import Image
import os

# 베이스 경로
base_dir = 'C:/cakd7/3차 프로젝트/원본데이터/'

# 변환이 필요한 파일이름 리스트 생성
filename_list = os.listdir(base_dir + '1. train_masks/')
# print(filename_list)

# 변환 후 저장될 디렉토리 생성
if not os.path.exists(base_dir + '2. train_masks_tiff'):
    os.mkdir(base_dir + '2. train_masks_tiff')
if not os.path.exists(base_dir + '2. train_imgs_tiff'):
    os.mkdir(base_dir + '2. train_imgs_tiff')


# 파일형식 변환
for filename in filename_list:
    # 새로운 파일 이름 생성
    new_filename = filename.split('.')[0]
    
    ## H&E염색 데이터 변환
    image = Image.open(base_dir + f'1. train_masks/{filename}')
    image.save(base_dir + f'2. train_masks_tiff/{new_filename}.tiff')
    
    ## 마스킹 데이터 변환
    image = Image.open(base_dir + f'1. train_imgs/{filename}')
    image.save(base_dir + f'2. train_imgs_tiff/{new_filename}.tiff')
    

['BC_01_0015.png', 'BC_01_0397.png', 'BC_01_0398.png', 'BC_01_0404.png', 'BC_01_0417.png', 'BC_01_0425.png', 'BC_01_0451.png', 'BC_01_0454.png', 'BC_01_0455.png', 'BC_01_0457.png', 'BC_01_0459.png', 'BC_01_0471.png', 'BC_01_0510.png', 'BC_01_0515.png', 'BC_01_0516.png', 'BC_01_0530.png', 'BC_01_0531.png', 'BC_01_0532.png', 'BC_01_0552.png', 'BC_01_0572.png', 'BC_01_0583.png', 'BC_01_0585.png', 'BC_01_0587.png', 'BC_01_0591.png', 'BC_01_0599.png', 'BC_01_0609.png', 'BC_01_0612.png', 'BC_01_0652.png', 'BC_01_0656.png', 'BC_01_0660.png', 'BC_01_0666.png', 'BC_01_0668.png', 'BC_01_0673.png', 'BC_01_0674.png', 'BC_01_0681.png', 'BC_01_0693.png', 'BC_01_0694.png', 'BC_01_0876.png', 'BC_01_0877.png', 'BC_01_0881.png', 'BC_01_0883.png', 'BC_01_0885.png', 'BC_01_0886.png', 'BC_01_0893.png', 'BC_01_0902.png', 'BC_01_0905.png', 'BC_01_0906.png', 'BC_01_0907.png', 'BC_01_0912.png', 'BC_01_0922.png', 'BC_01_0925.png', 'BC_01_0928.png', 'BC_01_0936.png', 'BC_01_0938.png', 'BC_01_0947.png', 'BC_01_09

### 이미지 내 전체 이미지 패치 추출 후 폴더에 파일 저장

In [ ]:
import os

# 베이스 경로
base_dir = 'C:/cakd7/3차 프로젝트/원본데이터/'

# 파일이름 리스트 생성
filename_list = os.listdir(base_dir + '2. train_masks_tiff/')


# 타일로 쪼개서 저장할 디렉토리 생성
if not os.path.exists(base_dir + '3. split_real_normal/'):
    os.mkdir(base_dir + '3. split_real_normal/')
if not os.path.exists(base_dir + '3. split_real_abnormal/'):
    os.mkdir(base_dir + '3. split_real_abnormal/')
if not os.path.exists(base_dir + '3. split_mask_normal/'):
    os.mkdir(base_dir + '3. split_mask_normal/')
if not os.path.exists(base_dir + '3. split_mask_abnormal/'):
    os.mkdir(base_dir + '3. split_mask_abnormal/')

# 타일 저장 경로 객체화
tile_real_normal_dir = base_dir + '3. split_real_normal/'
tile_real_abnormal_dir = base_dir + '3. split_real_abnormal/'
tile_mask_normal_dir = base_dir + '3. split_mask_normal/'
tile_mask_abnormal_dir = base_dir + '3. split_mask_abnormal/'


for filename in filename_list:   
    
    # 이미지패치가 저장될 폴더 생성
    if not os.path.exists(tile_real_normal_dir + filename):
        os.mkdir(tile_real_normal_dir + filename)
    if not os.path.exists(tile_real_abnormal_dir + filename):
        os.mkdir(tile_real_abnormal_dir + filename)
    if not os.path.exists(tile_mask_normal_dir + filename):
        os.mkdir(tile_mask_normal_dir + filename)
    if not os.path.exists(tile_mask_abnormal_dir + filename):
        os.mkdir(tile_mask_abnormal_dir + filename)
    
    
    # 오픈슬라이드 객체로 이미지 받아오기
    ## 마스킹
    slide_mask = openslide.open_slide(base_dir + '2. train_masks_tiff/' + filename)
    ## 원본
    slide_real = openslide.open_slide(base_dir + '2. train_imgs_tiff/' + filename)

    # 이미지 50*50 패치로 쪼개기
    ## 마스킹
    tiles_mask = DeepZoomGenerator(slide_mask, tile_size = 50, overlap = 0, limit_bounds = False)
    ## 원본
    tiles_real = DeepZoomGenerator(slide_real, tile_size = 50, overlap = 0, limit_bounds = False)
    
    
    # 전체 이미지 패치 해상도 레벨 확인
    mask_level = tiles_mask.level_count
    real_level = tiles_real.level_count
    if mask_level != real_level:
        print(f'{filename} 마스킹된 파일과 원본 이미지의 컬럼 해상도 레벨 다름')
    
    # 타일의 열개수, 행개수 객체화 후 확인
    cols_mask, rows_mask = tiles_mask.level_tiles[mask_level - 1]
    cols_real, rows_real = tiles_real.level_tiles[real_level - 1]
    if cols_mask != cols_real:
        print(f'{filename} 마스킹된 파일과 원본 이미지의 컬럼 타일 개수 다름')
    if rows_mask != rows_real:
        print(f'{filename} 마스킹된 파일과 원본 이미지의 로우 타일 개수 다름')
    
    #### 원본데이터와 마스킹 데이터 크기가 같아서 결론적으로 row, col개수가 같을 것으로 보임
    #### 해상도 레벨도 마찬가지
    
    print(f'****** 현재 {filename} 처리 중 ******')
    for row in range(rows_mask):
        for col in range(cols_mask):
            
            # 타일 저장 이름 정해주기
            tile_name_real_normal = os.path.join(tile_real_normal_dir + filename, f'{filename}_{row}_{col}')
            tile_name_real_abnormal = os.path.join(tile_real_abnormal_dir + filename, f'{filename}_{row}_{col}')
            tile_name_mask_normal = os.path.join(tile_mask_normal_dir + filename, f'{filename}_{row}_{col}')
            tile_name_mask_abnormal = os.path.join(tile_mask_abnormal_dir + filename, f'{filename}_{row}_{col}')
#             print('지금 저장 중인 파일: ', '\n', 'tile_name: ', row,"_", col)

            # 해당 컬럼, 로우에 해당하는 타일 객체화
            ## 마스킹
            temp_tile_mask = tiles_mask.get_tile(mask_level - 1, (col, row))
    #         print(temp_tile_mask)
            ## 원본
            temp_tile_real = tiles_real.get_tile(mask_level - 1, (col, row))
    #         print(temp_tile_real)

            temp_tile_real_RGB = temp_tile_real.convert('RGB')

            # 해당 타일 이미지를 3차원 배열화-> rgb로 3차원이나옴
            ## 마스킹
            temp_tile_mask_np = np.array(temp_tile_mask)
    #         print(temp_tile_mask_np.shape)
            ## 원본
            temp_tile_real_np = np.array(temp_tile_real_RGB)
    #         print(temp_tile_real_np.shape)


            # 타일의 로우, 컬럼 수 객체화
            tile_row, tile_col, dim = temp_tile_real_np.shape

            # 차원값 모두 255인(흰색부분)의 픽셀 수 및 비율 확인, 정상세포 픽셀 수 및 비율 확인
            white = 0
            normal = 0
            for i in range(tile_row):
                for j in range(tile_col):
                    if temp_tile_mask_np[i, j, 0] == 255 and temp_tile_mask_np[i, j, 1] == 255 and temp_tile_mask_np[i, j, 2] == 255:
                        white += 1

                    elif temp_tile_mask_np[i, j, 0] == 225 and temp_tile_mask_np[i, j, 1] == 225 and temp_tile_mask_np[i, j, 2] == 225:
                        normal += 1

            white_ratio = white / 2500
            normal_ratio = normal / 2500

#             print(white_ratio)
#             print(normal_ratio)

            if white_ratio > 0.3: 
                pass
            elif normal_ratio > 0.05 and tile_row == 50 and tile_col == 50 :
                # 저장하기
                plt.imsave(tile_name_real_normal + ".png", temp_tile_real_np)
                plt.imsave(tile_name_mask_normal + ".png", temp_tile_mask_np)

            elif tile_row == 50 and tile_col == 50:
                # 저장하기
                plt.imsave(tile_name_real_abnormal + ".png", temp_tile_real_np)
                plt.imsave(tile_name_mask_abnormal + ".png", temp_tile_mask_np)

****** 현재 BC_01_0015.tiff 처리 중 ******
****** 현재 BC_01_0397.tiff 처리 중 ******
****** 현재 BC_01_0398.tiff 처리 중 ******
****** 현재 BC_01_0404.tiff 처리 중 ******
****** 현재 BC_01_0417.tiff 처리 중 ******
****** 현재 BC_01_0425.tiff 처리 중 ******
****** 현재 BC_01_0451.tiff 처리 중 ******
****** 현재 BC_01_0454.tiff 처리 중 ******
****** 현재 BC_01_0455.tiff 처리 중 ******
****** 현재 BC_01_0457.tiff 처리 중 ******
****** 현재 BC_01_0459.tiff 처리 중 ******
****** 현재 BC_01_0471.tiff 처리 중 ******
****** 현재 BC_01_0510.tiff 처리 중 ******
****** 현재 BC_01_0515.tiff 처리 중 ******
****** 현재 BC_01_0516.tiff 처리 중 ******
****** 현재 BC_01_0530.tiff 처리 중 ******
****** 현재 BC_01_0531.tiff 처리 중 ******
****** 현재 BC_01_0532.tiff 처리 중 ******
****** 현재 BC_01_0552.tiff 처리 중 ******
****** 현재 BC_01_0572.tiff 처리 중 ******
****** 현재 BC_01_0583.tiff 처리 중 ******
****** 현재 BC_01_0585.tiff 처리 중 ******
****** 현재 BC_01_0587.tiff 처리 중 ******
****** 현재 BC_01_0591.tiff 처리 중 ******
****** 현재 BC_01_0599.tiff 처리 중 ******
****** 현재 BC_01_0609.tiff 처리 중 ******
****** 현재 BC

## 전체 데이터 진행

### png 파일을 tiff파일로 변환

In [ ]:
from PIL import Image
import os

# 베이스 경로
base_dir = 'C:/cakd7/3차 프로젝트/원본데이터/'


# 변환이 필요한 파일이름 리스트 생성
no_filename_list = os.listdir(base_dir + '1. train_masks/')
filename_list = os.listdir(base_dir + '1. train_imgs/')
for filename in no_filename_list:
    filename_list.remove(filename)
# print(filename_list)


# 변환 후 저장될 디렉토리 생성
if not os.path.exists(base_dir + '4. train_imgs_tiff'):
    os.mkdir(base_dir + '4. train_imgs_tiff')


# 파일형식 변환
for filename in filename_list:
    # 새로운 파일 이름 생성
    new_filename = filename.split('.')[0]
    
    ## H&E염색 데이터 변환
    image = Image.open(base_dir + f'1. train_imgs/{filename}')
    image.save(base_dir + f'4. train_imgs_tiff/{new_filename}.tiff')

    

### 이미지 타일 추출 후 폴더에 파일 저장

In [ ]:
import os

# 베이스 경로
base_dir = 'C:/cakd7/3차 프로젝트/원본데이터/'

# 전체 파일이름 리스트 생성
filename_list = os.listdir(base_dir + '4. train_imgs_tiff/')


# 타일로 쪼개서 저장할 디렉토리 생성
if not os.path.exists(base_dir + '5. split_train_tiff/'):
    os.mkdir(base_dir + '5. split_train_tiff/')


# 타일 저장 경로 객체화
tile_real_dir = base_dir + '5. split_train_tiff/'


for filename in filename_list[20:]:   
    
    # 이미지패치가 저장될 폴더 생성
    if not os.path.exists(tile_real_dir + filename):
        os.mkdir(tile_real_dir + filename)
    
    
    # 오픈슬라이드 객체로 이미지 받아오기
    ## 원본
    slide_real = openslide.open_slide(base_dir + '4. train_imgs_tiff/' + filename)

    # 이미지 50*50 패치로 쪼개기
    ## 원본
    tiles_real = DeepZoomGenerator(slide_real, tile_size = 50, overlap = 0, limit_bounds = False)

    
    # 전체 이미지 패치 해상도 레벨 확인
    real_level = tiles_real.level_count
    
    # 타일의 열개수, 행개수 객체화 후 확인
    cols_real, rows_real = tiles_real.level_tiles[real_level - 1]
    
    
    print(f'****** 현재 {filename} 처리 중 ******')
    for row in range(rows_real):
        for col in range(cols_real):
            
            # 타일 저장 이름 정해주기
            tile_name_real = os.path.join(tile_real_dir + filename, f'{filename}_{col}_{row}')
#             print('지금 저장 중인 파일: ', '\n', 'tile_name: ', row,"_", col)

            # 해당 컬럼, 로우에 해당하는 타일 객체화
            ## 원본
            temp_tile_real = tiles_real.get_tile(real_level - 1, (col, row))
    #         print(temp_tile_real)

            temp_tile_real_RGB = temp_tile_real.convert('RGB')

            # 해당 타일 이미지를 3차원 배열화-> rgb로 3차원이 나옴
            ## 원본
            temp_tile_real_np = np.array(temp_tile_real_RGB)
    #         print(temp_tile_real_np.shape)

            # 타일의 로우, 컬럼 수 객체화
            tile_row, tile_col, dim = temp_tile_real_np.shape

            # 차원값 모두 255인(흰색부분)의 픽셀 수 및 비율 확인, 정상세포 픽셀 수 및 비율 확인
            white = 0
            for i in range(tile_row):
                for j in range(tile_col):
                    if temp_tile_real_np[i, j, 0] in   [242,243,244] and temp_tile_real_np[i, j, 1] in [242,243,244] and temp_tile_real_np[i, j, 2] in [242,243,244]:
                        white += 1
            white_ratio = white / 2500


            if white_ratio > 0.3: 
                pass
            elif tile_row == 50 and tile_col == 50 :
                # 저장하기
                plt.imsave(tile_name_real + '.png', temp_tile_real_np)


****** 현재 BC_01_0217.tiff 처리 중 ******
****** 현재 BC_01_0218.tiff 처리 중 ******
****** 현재 BC_01_0223.tiff 처리 중 ******
****** 현재 BC_01_0224.tiff 처리 중 ******
****** 현재 BC_01_0226.tiff 처리 중 ******
****** 현재 BC_01_0227.tiff 처리 중 ******
****** 현재 BC_01_0229.tiff 처리 중 ******
****** 현재 BC_01_0230.tiff 처리 중 ******
****** 현재 BC_01_0232.tiff 처리 중 ******
****** 현재 BC_01_0234.tiff 처리 중 ******
****** 현재 BC_01_0238.tiff 처리 중 ******
****** 현재 BC_01_0240.tiff 처리 중 ******
****** 현재 BC_01_0245.tiff 처리 중 ******
****** 현재 BC_01_0246.tiff 처리 중 ******
****** 현재 BC_01_0249.tiff 처리 중 ******
****** 현재 BC_01_0250.tiff 처리 중 ******
****** 현재 BC_01_0252.tiff 처리 중 ******
****** 현재 BC_01_0254.tiff 처리 중 ******
****** 현재 BC_01_0255.tiff 처리 중 ******
****** 현재 BC_01_0257.tiff 처리 중 ******
****** 현재 BC_01_0261.tiff 처리 중 ******
****** 현재 BC_01_0262.tiff 처리 중 ******
****** 현재 BC_01_0263.tiff 처리 중 ******
****** 현재 BC_01_0264.tiff 처리 중 ******
****** 현재 BC_01_0268.tiff 처리 중 ******
****** 현재 BC_01_0270.tiff 처리 중 ******
****** 현재 BC

****** 현재 BC_01_1414.tiff 처리 중 ******
****** 현재 BC_01_1416.tiff 처리 중 ******
****** 현재 BC_01_1417.tiff 처리 중 ******
****** 현재 BC_01_1418.tiff 처리 중 ******
****** 현재 BC_01_1419.tiff 처리 중 ******
****** 현재 BC_01_1435.tiff 처리 중 ******
****** 현재 BC_01_1436.tiff 처리 중 ******
****** 현재 BC_01_1442.tiff 처리 중 ******
****** 현재 BC_01_1468.tiff 처리 중 ******
****** 현재 BC_01_1470.tiff 처리 중 ******
****** 현재 BC_01_1486.tiff 처리 중 ******
****** 현재 BC_01_1495.tiff 처리 중 ******
****** 현재 BC_01_1496.tiff 처리 중 ******
****** 현재 BC_01_1503.tiff 처리 중 ******
****** 현재 BC_01_1509.tiff 처리 중 ******
****** 현재 BC_01_1519.tiff 처리 중 ******
****** 현재 BC_01_1520.tiff 처리 중 ******
****** 현재 BC_01_1526.tiff 처리 중 ******
****** 현재 BC_01_1527.tiff 처리 중 ******
****** 현재 BC_01_1529.tiff 처리 중 ******
****** 현재 BC_01_1533.tiff 처리 중 ******
****** 현재 BC_01_1539.tiff 처리 중 ******
****** 현재 BC_01_1541.tiff 처리 중 ******
****** 현재 BC_01_1552.tiff 처리 중 ******
****** 현재 BC_01_1557.tiff 처리 중 ******
****** 현재 BC_01_1560.tiff 처리 중 ******
****** 현재 BC

****** 현재 BC_01_2358.tiff 처리 중 ******
****** 현재 BC_01_2359.tiff 처리 중 ******
****** 현재 BC_01_2361.tiff 처리 중 ******
****** 현재 BC_01_2364.tiff 처리 중 ******
****** 현재 BC_01_2373.tiff 처리 중 ******
****** 현재 BC_01_2376.tiff 처리 중 ******
****** 현재 BC_01_2377.tiff 처리 중 ******
****** 현재 BC_01_2381.tiff 처리 중 ******
****** 현재 BC_01_2382.tiff 처리 중 ******
****** 현재 BC_01_2383.tiff 처리 중 ******
****** 현재 BC_01_2384.tiff 처리 중 ******
****** 현재 BC_01_2385.tiff 처리 중 ******
****** 현재 BC_01_2388.tiff 처리 중 ******
****** 현재 BC_01_2390.tiff 처리 중 ******
****** 현재 BC_01_2399.tiff 처리 중 ******
****** 현재 BC_01_2404.tiff 처리 중 ******
****** 현재 BC_01_2407.tiff 처리 중 ******
****** 현재 BC_01_2408.tiff 처리 중 ******
****** 현재 BC_01_2429.tiff 처리 중 ******
****** 현재 BC_01_2436.tiff 처리 중 ******
****** 현재 BC_01_2437.tiff 처리 중 ******
****** 현재 BC_01_2439.tiff 처리 중 ******
****** 현재 BC_01_2446.tiff 처리 중 ******
****** 현재 BC_01_2449.tiff 처리 중 ******
****** 현재 BC_01_2456.tiff 처리 중 ******
****** 현재 BC_01_2461.tiff 처리 중 ******
****** 현재 BC

****** 현재 BC_01_2808.tiff 처리 중 ******
****** 현재 BC_01_2809.tiff 처리 중 ******
****** 현재 BC_01_2811.tiff 처리 중 ******
****** 현재 BC_01_2812.tiff 처리 중 ******
****** 현재 BC_01_2813.tiff 처리 중 ******
****** 현재 BC_01_2815.tiff 처리 중 ******
****** 현재 BC_01_2816.tiff 처리 중 ******
****** 현재 BC_01_2817.tiff 처리 중 ******
****** 현재 BC_01_2819.tiff 처리 중 ******
****** 현재 BC_01_2820.tiff 처리 중 ******
****** 현재 BC_01_2821.tiff 처리 중 ******
****** 현재 BC_01_2822.tiff 처리 중 ******
****** 현재 BC_01_2823.tiff 처리 중 ******
****** 현재 BC_01_2824.tiff 처리 중 ******
****** 현재 BC_01_2825.tiff 처리 중 ******
****** 현재 BC_01_2826.tiff 처리 중 ******
****** 현재 BC_01_2827.tiff 처리 중 ******
****** 현재 BC_01_2829.tiff 처리 중 ******
****** 현재 BC_01_2830.tiff 처리 중 ******
****** 현재 BC_01_2831.tiff 처리 중 ******
****** 현재 BC_01_2832.tiff 처리 중 ******
****** 현재 BC_01_2833.tiff 처리 중 ******
****** 현재 BC_01_2834.tiff 처리 중 ******
****** 현재 BC_01_2835.tiff 처리 중 ******
****** 현재 BC_01_2846.tiff 처리 중 ******
****** 현재 BC_01_2847.tiff 처리 중 ******
****** 현재 BC

****** 현재 BC_01_3144.tiff 처리 중 ******
****** 현재 BC_01_3150.tiff 처리 중 ******
****** 현재 BC_01_3155.tiff 처리 중 ******
****** 현재 BC_01_3156.tiff 처리 중 ******
****** 현재 BC_01_3157.tiff 처리 중 ******
****** 현재 BC_01_3169.tiff 처리 중 ******
****** 현재 BC_01_3171.tiff 처리 중 ******
****** 현재 BC_01_3172.tiff 처리 중 ******
****** 현재 BC_01_3174.tiff 처리 중 ******
****** 현재 BC_01_3179.tiff 처리 중 ******
****** 현재 BC_01_3183.tiff 처리 중 ******
****** 현재 BC_01_3192.tiff 처리 중 ******
****** 현재 BC_01_3195.tiff 처리 중 ******
****** 현재 BC_01_3199.tiff 처리 중 ******
****** 현재 BC_01_3200.tiff 처리 중 ******
****** 현재 BC_01_3206.tiff 처리 중 ******
****** 현재 BC_01_3208.tiff 처리 중 ******
****** 현재 BC_01_3213.tiff 처리 중 ******
****** 현재 BC_01_3217.tiff 처리 중 ******
****** 현재 BC_01_3219.tiff 처리 중 ******
****** 현재 BC_01_3232.tiff 처리 중 ******
****** 현재 BC_01_3235.tiff 처리 중 ******
****** 현재 BC_01_3242.tiff 처리 중 ******
****** 현재 BC_01_3243.tiff 처리 중 ******
****** 현재 BC_01_3244.tiff 처리 중 ******
****** 현재 BC_01_3246.tiff 처리 중 ******
****** 현재 BC